In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd ..
%cd /content/drive/MyDrive/BrontoByte/Differential-Privacy/federated_learning_web_attack_detection

Mounted at /content/drive
/
/content/drive/MyDrive/BrontoByte/Differential-Privacy/federated_learning_web_attack_detection


In [2]:
!pip install gmpy2
!pip install tensorflow_privacy
!pip install functorch
!pip install opacus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 KB 18.9 MB/s eta 0:00:00

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 887.1/887.4 MB 73.4 MB/s eta 0:00:01tcmalloc: large alloc 1109270528 bytes == 0x38eb0000 @  0x7f9a3350d615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 KB 42.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0

In [6]:
import json

config = {
    'global_config': {
        'name': 'Test', # Tên của thực nghiệm,
        'overwrite_experiment': True, # Ghi đè lên folder thực nghiệm cũ
        'device': 'cuda', 
        'noise_level': 0.0001, # Mức nhiễu thêm vào local model ở client trước khi gửi đến server.
        'compress_digits': 15, # Số chữ số sau dấu phẩy mà local model ở client nén.
        'dp_mode': True # Sử dụng differential privacy
    }, 
    'data_config': { 
        'dataset_name': 'csic2010', # Dataset sử dụng
        'data_sampling_technique': 'iid' # Cách chia data cho các client
    },
    'fed_config': {
        'global_epochs': 10, # Số lượng vòng truyền thông.
        'local_epochs': 1, # Số lượng epoch huấn luyện tại local.
        'nb_clients': 5, # Số lượng client tham gia training.
        'fraction': 1.0, # Tỷ lệ client tham gia training tại mỗi vòng.
        'batch_size': 64 # batch size cho training tại mỗi client.
    }, 
    'optim_config': {
        'lr': 0.001 # learning rate
    }, 
    'dp_config': {
        #Privacy Parameters for (epsilon, delta)-DP
        'epsilon': 50, 
        'delta': 3e-5, 
        'clipping_norm': 2.0,
        'is_fixed_client_iter': True,
        'client_iter': 700
    }
}

config_json_object = json.dumps(config, indent=4)

with open("config.json", "w") as outfile:
    outfile.write(config_json_object)

In [ ]:
!python main.py -cf config.json

*** FIX VER 11
Overwrite experiment mode:  True
----------------------------------------------------------------------------------------------------
[INFO] DATA INFORMATION
Using csic2010 dataset ...
+++ csic2010 dataset: +++
	Number of normal requests:  18640
	Number of anomalous requests:  15873
	Number of total requests:  34513


In [ ]:
from opacus.accountants.utils import get_noise_multiplier

batch_size = 64
SR = batch_size / 5600
E = 10
print(get_noise_multiplier(
            target_epsilon=12,
            target_delta=1e-4,
            sample_rate=SR,
            epochs=E,
        ))

In [3]:
import numpy as np

mtx_size = 5
server_decoded_message = {
            'R': None, 'S': None,
            'M': np.zeros((mtx_size, mtx_size)),
            'N': np.zeros((mtx_size, mtx_size))
        }

server_decoded_message['R'] = 1

In [4]:
print(server_decoded_message)

{'R': 1, 'S': None, 'M': array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]]), 'N': array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])}
